# Day 5: Week 1 Review - Capital Gains Comprehensive Review

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/astoreyai/money-talks/blob/main/class4_taxes_portfolio/week1_capital_gains/day05_week1_review.ipynb)

## Class 4: Taxes & Portfolio Maintenance
### Week 1: Capital Gains - Day 5 of 20

---

## Learning Objectives

By the end of this lesson, you will be able to:
1. **Synthesize** all capital gains concepts from Week 1
2. **Apply** tax calculations to real portfolio scenarios
3. **Build** a comprehensive capital gains tax calculator
4. **Optimize** selling decisions for tax efficiency
5. **Create** a year-end tax planning checklist

---

## Lecture: Week 1 Comprehensive Review (30 min)

### Week 1 Concepts Summary

```
WEEK 1: CAPITAL GAINS MASTERY
=============================

Day 1: Investment Tax Fundamentals
├── Three income types: Gains, Dividends, Interest
├── Realized vs Unrealized gains
├── Tax deferral benefits
└── Key tax forms (1099-B, Schedule D)

Day 2: Short-Term vs Long-Term
├── Holding period: >365 days = LT
├── ST rates: 10%-37% (ordinary)
├── LT rates: 0%, 15%, 20%
└── 0% bracket opportunity

Day 3: Cost Basis Methods
├── FIFO (default)
├── LIFO
├── Specific Identification
└── Average Cost (mutual funds)

Day 4: Capital Losses
├── Loss netting rules
├── $3,000 ordinary income deduction
├── Carryforward rules
└── Wash sale rule (61-day window)
```

### Master Formula Reference

```
ESSENTIAL FORMULAS
==================

CAPITAL GAIN/LOSS:
──────────────────
Gain/Loss = Sale Proceeds - Cost Basis

Cost Basis = Purchase Price + Commissions + Reinvested Dividends
             - Return of Capital Distributions


AFTER-TAX RETURN:
─────────────────
After-Tax Return = Pre-Tax Return × (1 - Tax Rate)


HOLDING PERIOD:
───────────────
Days Held = Sale Date - Purchase Date
Long-Term if Days > 365


LOSS NETTING:
─────────────
Net ST = ST Gains - ST Losses
Net LT = LT Gains - LT Losses
If both negative: Deduct min($3,000, Total Loss)
Carryforward = Total Loss - $3,000


WASH SALE ADJUSTMENT:
─────────────────────
New Basis = Purchase Price + Disallowed Loss
```

### Tax Rate Quick Reference

```
2024 TAX RATES SUMMARY
======================

ORDINARY INCOME (ST Gains):
───────────────────────────
Single              Married Joint       Rate
$0-$11,600          $0-$23,200          10%
$11,601-$47,150     $23,201-$94,300     12%
$47,151-$100,525    $94,301-$201,050    22%
$100,526-$191,950   $201,051-$383,900   24%
$191,951-$243,725   $383,901-$487,450   32%
$243,726-$609,350   $487,451-$731,200   35%
$609,351+           $731,201+           37%


LONG-TERM CAPITAL GAINS:
────────────────────────
Single              Married Joint       Rate
$0-$47,025          $0-$94,050          0%
$47,026-$518,900    $94,051-$583,750    15%
$518,901+           $583,751+           20%

NIIT (3.8%) applies above $200K/$250K AGI


QUALIFIED DIVIDENDS:
────────────────────
Same rates as LTCG (0%, 15%, 20%)
```

### Year-End Tax Planning Checklist

```
YEAR-END TAX OPTIMIZATION CHECKLIST
===================================

□ REVIEW REALIZED GAINS/LOSSES
  ├── Calculate YTD net gains by type (ST/LT)
  └── Identify tax liability estimate

□ HARVEST TAX LOSSES
  ├── Find positions with unrealized losses
  ├── Prioritize ST losses (offset higher-taxed gains)
  └── Avoid wash sales (wait 31 days or swap)

□ CHECK HOLDING PERIODS
  ├── Positions approaching 1-year mark
  ├── Delay selling until LT if possible
  └── Calculate breakeven for waiting

□ OPTIMIZE COST BASIS
  ├── Review available lots before selling
  ├── Use specific ID for tax efficiency
  └── Verify DRIP basis is tracked

□ USE 0% BRACKET (if applicable)
  ├── Calculate space in 0% LTCG bracket
  ├── Harvest gains tax-free
  └── Reset basis higher for future

□ DOCUMENT CARRYFORWARDS
  ├── Track unused losses
  └── Note ST vs LT character

□ ESTIMATED TAX PAYMENTS
  ├── Calculate if underpaid
  └── Make Q4 payment if needed
```

---

## Hands-On Practice: Comprehensive Tax Calculator (15 min)

Let's build a complete capital gains tax analysis system.

In [ ]:
# Install and import required libraries
!pip install pandas numpy matplotlib -q

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

print("Day 5: Week 1 Review - Capital Gains")
print("="*50)

In [ ]:
# Comprehensive Capital Gains Tax Calculator
class CapitalGainsTaxCalculator:
    """
    Complete capital gains tax calculator integrating all Week 1 concepts.
    """
    
    def __init__(self, filing_status='single', ordinary_income=0):
        self.filing_status = filing_status
        self.ordinary_income = ordinary_income
        self.transactions = []
        self.lots = {}  # ticker -> list of lots
        self.carryforward_st = 0
        self.carryforward_lt = 0
        
        # 2024 tax brackets
        self.ltcg_brackets = {
            'single': [(47025, 0.00), (518900, 0.15), (float('inf'), 0.20)],
            'married': [(94050, 0.00), (583750, 0.15), (float('inf'), 0.20)]
        }
        
        self.ordinary_brackets = {
            'single': [
                (11600, 0.10), (47150, 0.12), (100525, 0.22),
                (191950, 0.24), (243725, 0.32), (609350, 0.35),
                (float('inf'), 0.37)
            ],
            'married': [
                (23200, 0.10), (94300, 0.12), (201050, 0.22),
                (383900, 0.24), (487450, 0.32), (731200, 0.35),
                (float('inf'), 0.37)
            ]
        }
    
    def add_purchase(self, ticker, date, shares, price, commission=0):
        """Record a stock purchase."""
        if isinstance(date, str):
            date = datetime.strptime(date, '%Y-%m-%d')
        
        total_cost = (shares * price) + commission
        
        if ticker not in self.lots:
            self.lots[ticker] = []
        
        self.lots[ticker].append({
            'date': date,
            'shares': shares,
            'cost_per_share': total_cost / shares,
            'total_cost': total_cost,
            'remaining': shares
        })
    
    def record_sale(self, ticker, date, shares, price, method='fifo'):
        """Record a stock sale and calculate gain/loss."""
        if isinstance(date, str):
            sale_date = datetime.strptime(date, '%Y-%m-%d')
        else:
            sale_date = date
        
        proceeds = shares * price
        remaining = shares
        total_basis = 0
        st_gain = 0
        lt_gain = 0
        
        # Sort lots based on method
        if method == 'fifo':
            sorted_lots = sorted(self.lots.get(ticker, []), key=lambda x: x['date'])
        elif method == 'lifo':
            sorted_lots = sorted(self.lots.get(ticker, []), key=lambda x: x['date'], reverse=True)
        else:
            sorted_lots = self.lots.get(ticker, [])
        
        for lot in sorted_lots:
            if remaining <= 0:
                break
            if lot['remaining'] <= 0:
                continue
            
            shares_from_lot = min(lot['remaining'], remaining)
            basis_from_lot = shares_from_lot * lot['cost_per_share']
            proceeds_from_lot = shares_from_lot * price
            gain = proceeds_from_lot - basis_from_lot
            
            days_held = (sale_date - lot['date']).days
            is_lt = days_held > 365
            
            if is_lt:
                lt_gain += gain
            else:
                st_gain += gain
            
            total_basis += basis_from_lot
            lot['remaining'] -= shares_from_lot
            remaining -= shares_from_lot
        
        self.transactions.append({
            'ticker': ticker,
            'date': sale_date,
            'shares': shares,
            'proceeds': proceeds,
            'basis': total_basis,
            'st_gain': st_gain,
            'lt_gain': lt_gain,
            'total_gain': st_gain + lt_gain
        })
        
        return {
            'proceeds': proceeds,
            'basis': total_basis,
            'st_gain': st_gain,
            'lt_gain': lt_gain,
            'total_gain': st_gain + lt_gain
        }
    
    def get_marginal_rate(self, income):
        """Get marginal ordinary income tax rate."""
        for limit, rate in self.ordinary_brackets[self.filing_status]:
            if income <= limit:
                return rate
        return self.ordinary_brackets[self.filing_status][-1][1]
    
    def get_ltcg_rate(self, income):
        """Get long-term capital gains rate."""
        for limit, rate in self.ltcg_brackets[self.filing_status]:
            if income <= limit:
                return rate
        return self.ltcg_brackets[self.filing_status][-1][1]
    
    def calculate_year_tax(self):
        """Calculate total capital gains tax for the year."""
        # Sum gains by type
        total_st_gain = sum(t['st_gain'] for t in self.transactions if t['st_gain'] > 0)
        total_st_loss = sum(abs(t['st_gain']) for t in self.transactions if t['st_gain'] < 0)
        total_lt_gain = sum(t['lt_gain'] for t in self.transactions if t['lt_gain'] > 0)
        total_lt_loss = sum(abs(t['lt_gain']) for t in self.transactions if t['lt_gain'] < 0)
        
        # Apply carryforwards
        total_st_loss += self.carryforward_st
        total_lt_loss += self.carryforward_lt
        
        # Net within categories
        net_st = total_st_gain - total_st_loss
        net_lt = total_lt_gain - total_lt_loss
        
        # Cross-netting
        taxable_st = 0
        taxable_lt = 0
        ordinary_deduction = 0
        new_carryforward_st = 0
        new_carryforward_lt = 0
        
        if net_st >= 0 and net_lt >= 0:
            taxable_st = net_st
            taxable_lt = net_lt
        elif net_st < 0 and net_lt < 0:
            total_loss = abs(net_st) + abs(net_lt)
            ordinary_deduction = min(total_loss, 3000)
            carryforward = total_loss - ordinary_deduction
            # Allocate carryforward proportionally
            if total_loss > 0:
                new_carryforward_st = carryforward * (abs(net_st) / total_loss)
                new_carryforward_lt = carryforward * (abs(net_lt) / total_loss)
        elif net_st < 0:
            # ST loss offsets LT gain
            combined = net_lt + net_st
            if combined >= 0:
                taxable_lt = combined
            else:
                ordinary_deduction = min(abs(combined), 3000)
                new_carryforward_st = max(0, abs(combined) - ordinary_deduction)
        else:
            # LT loss offsets ST gain
            combined = net_st + net_lt
            if combined >= 0:
                taxable_st = combined
            else:
                ordinary_deduction = min(abs(combined), 3000)
                new_carryforward_lt = max(0, abs(combined) - ordinary_deduction)
        
        # Calculate taxes
        total_income = self.ordinary_income + taxable_st
        st_rate = self.get_marginal_rate(total_income)
        lt_rate = self.get_ltcg_rate(total_income + taxable_lt)
        
        st_tax = taxable_st * st_rate
        lt_tax = taxable_lt * lt_rate
        ordinary_savings = ordinary_deduction * self.get_marginal_rate(self.ordinary_income)
        
        return {
            'total_st_gains': total_st_gain,
            'total_st_losses': total_st_loss,
            'total_lt_gains': total_lt_gain,
            'total_lt_losses': total_lt_loss,
            'net_st': net_st,
            'net_lt': net_lt,
            'taxable_st': taxable_st,
            'taxable_lt': taxable_lt,
            'st_rate': st_rate,
            'lt_rate': lt_rate,
            'st_tax': st_tax,
            'lt_tax': lt_tax,
            'total_cap_gains_tax': st_tax + lt_tax,
            'ordinary_deduction': ordinary_deduction,
            'ordinary_tax_savings': ordinary_savings,
            'carryforward_st': new_carryforward_st,
            'carryforward_lt': new_carryforward_lt
        }
    
    def get_unrealized(self, current_prices):
        """Calculate unrealized gains/losses."""
        unrealized = []
        today = datetime.now()
        
        for ticker, lots in self.lots.items():
            current_price = current_prices.get(ticker, 0)
            
            for i, lot in enumerate(lots):
                if lot['remaining'] > 0:
                    market_value = lot['remaining'] * current_price
                    cost_basis = lot['remaining'] * lot['cost_per_share']
                    gain = market_value - cost_basis
                    days = (today - lot['date']).days
                    
                    unrealized.append({
                        'Ticker': ticker,
                        'Lot': i + 1,
                        'Shares': lot['remaining'],
                        'Cost Basis': cost_basis,
                        'Market Value': market_value,
                        'Unrealized Gain': gain,
                        'Days Held': days,
                        'Type': 'LT' if days > 365 else 'ST'
                    })
        
        return pd.DataFrame(unrealized)

print("CapitalGainsTaxCalculator class defined!")

In [ ]:
# Comprehensive Example
print("COMPREHENSIVE CAPITAL GAINS TAX CALCULATION")
print("="*60)

# Create calculator for single filer with $80K income
calc = CapitalGainsTaxCalculator(filing_status='single', ordinary_income=80000)

# Add purchase history
print("\n1. RECORDING PURCHASES:")
print("-"*40)
calc.add_purchase('AAPL', '2023-01-15', 100, 150)
print("AAPL: 100 shares @ $150 on 2023-01-15")

calc.add_purchase('AAPL', '2023-07-20', 50, 180)
print("AAPL: 50 shares @ $180 on 2023-07-20")

calc.add_purchase('MSFT', '2023-03-10', 40, 280)
print("MSFT: 40 shares @ $280 on 2023-03-10")

calc.add_purchase('GOOGL', '2024-02-01', 30, 140)
print("GOOGL: 30 shares @ $140 on 2024-02-01")

calc.add_purchase('META', '2024-06-15', 25, 500)
print("META: 25 shares @ $500 on 2024-06-15")

# Record sales
print("\n2. RECORDING SALES (2024):")
print("-"*40)

# Sale 1: AAPL (long-term gain)
sale1 = calc.record_sale('AAPL', '2024-09-01', 80, 220, method='fifo')
print(f"AAPL: Sold 80 shares @ $220")
print(f"  Basis: ${sale1['basis']:,.0f}, Gain: ${sale1['total_gain']:,.0f} (LT: ${sale1['lt_gain']:,.0f})")

# Sale 2: MSFT (long-term gain)
sale2 = calc.record_sale('MSFT', '2024-10-15', 40, 420, method='fifo')
print(f"MSFT: Sold 40 shares @ $420")
print(f"  Basis: ${sale2['basis']:,.0f}, Gain: ${sale2['total_gain']:,.0f} (LT: ${sale2['lt_gain']:,.0f})")

# Sale 3: GOOGL (short-term gain)
sale3 = calc.record_sale('GOOGL', '2024-08-20', 30, 170, method='fifo')
print(f"GOOGL: Sold 30 shares @ $170")
print(f"  Basis: ${sale3['basis']:,.0f}, Gain: ${sale3['total_gain']:,.0f} (ST: ${sale3['st_gain']:,.0f})")

# Sale 4: META (short-term loss)
sale4 = calc.record_sale('META', '2024-11-01', 25, 450, method='fifo')
print(f"META: Sold 25 shares @ $450")
print(f"  Basis: ${sale4['basis']:,.0f}, Gain: ${sale4['total_gain']:,.0f} (ST: ${sale4['st_gain']:,.0f})")

In [ ]:
# Calculate Year-End Taxes
print("\n3. YEAR-END TAX CALCULATION:")
print("="*60)

tax_result = calc.calculate_year_tax()

print("\nGains and Losses Summary:")
print(f"  Total ST Gains:  ${tax_result['total_st_gains']:,.0f}")
print(f"  Total ST Losses: ${tax_result['total_st_losses']:,.0f}")
print(f"  Total LT Gains:  ${tax_result['total_lt_gains']:,.0f}")
print(f"  Total LT Losses: ${tax_result['total_lt_losses']:,.0f}")

print("\nNetting Results:")
print(f"  Net ST: ${tax_result['net_st']:,.0f}")
print(f"  Net LT: ${tax_result['net_lt']:,.0f}")

print("\nTaxable Amounts:")
print(f"  Taxable ST Gain: ${tax_result['taxable_st']:,.0f} @ {tax_result['st_rate']*100:.0f}%")
print(f"  Taxable LT Gain: ${tax_result['taxable_lt']:,.0f} @ {tax_result['lt_rate']*100:.0f}%")

print("\nTax Liability:")
print(f"  ST Capital Gains Tax: ${tax_result['st_tax']:,.0f}")
print(f"  LT Capital Gains Tax: ${tax_result['lt_tax']:,.0f}")
print(f"  ─────────────────────────────")
print(f"  TOTAL CAP GAINS TAX: ${tax_result['total_cap_gains_tax']:,.0f}")

if tax_result['ordinary_deduction'] > 0:
    print(f"\n  Ordinary Income Deduction: ${tax_result['ordinary_deduction']:,.0f}")
    print(f"  Tax Savings from Deduction: ${tax_result['ordinary_tax_savings']:,.0f}")

if tax_result['carryforward_st'] + tax_result['carryforward_lt'] > 0:
    print(f"\nCarryforward to Next Year:")
    print(f"  ST Loss: ${tax_result['carryforward_st']:,.0f}")
    print(f"  LT Loss: ${tax_result['carryforward_lt']:,.0f}")

In [ ]:
# Visualize Tax Breakdown
def visualize_tax_summary(tax_result):
    """Create visual summary of capital gains taxes."""
    
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    
    # Plot 1: Gains vs Losses
    categories = ['ST Gains', 'ST Losses', 'LT Gains', 'LT Losses']
    values = [
        tax_result['total_st_gains'],
        tax_result['total_st_losses'],
        tax_result['total_lt_gains'],
        tax_result['total_lt_losses']
    ]
    colors = ['green', 'red', 'lightgreen', 'lightcoral']
    
    axes[0].bar(categories, values, color=colors)
    axes[0].set_ylabel('Amount ($)')
    axes[0].set_title('Gains vs Losses by Type')
    axes[0].tick_params(axis='x', rotation=45)
    for i, v in enumerate(values):
        axes[0].annotate(f'${v:,.0f}', xy=(i, v), ha='center', va='bottom')
    
    # Plot 2: Taxable Amount Breakdown
    taxable = [tax_result['taxable_st'], tax_result['taxable_lt']]
    labels = ['Short-Term\n(Ordinary Rates)', 'Long-Term\n(Preferential Rates)']
    colors = ['coral', 'steelblue']
    
    if sum(taxable) > 0:
        axes[1].pie(taxable, labels=labels, colors=colors, autopct='%1.1f%%',
                   startangle=90, explode=(0.05, 0))
        axes[1].set_title(f'Taxable Gains: ${sum(taxable):,.0f}')
    else:
        axes[1].text(0.5, 0.5, 'No Taxable\nGains', ha='center', va='center', fontsize=14)
        axes[1].set_title('Taxable Gains: $0')
    
    # Plot 3: Tax Due
    tax_components = ['ST Tax', 'LT Tax']
    tax_amounts = [tax_result['st_tax'], tax_result['lt_tax']]
    
    bars = axes[2].bar(tax_components, tax_amounts, color=['coral', 'steelblue'])
    axes[2].set_ylabel('Tax ($)')
    axes[2].set_title(f'Total Tax Due: ${tax_result["total_cap_gains_tax"]:,.0f}')
    
    for bar, amt in zip(bars, tax_amounts):
        axes[2].annotate(f'${amt:,.0f}', xy=(bar.get_x() + bar.get_width()/2, amt),
                        ha='center', va='bottom')
    
    plt.tight_layout()
    plt.show()

visualize_tax_summary(tax_result)

In [ ]:
# Unrealized Gains Analysis
print("\n4. UNREALIZED POSITIONS:")
print("="*60)

current_prices = {'AAPL': 225, 'MSFT': 430, 'GOOGL': 175, 'META': 480}
unrealized = calc.get_unrealized(current_prices)

if len(unrealized) > 0:
    print(unrealized.to_string(index=False))
    
    print("\nSummary:")
    total_unrealized = unrealized['Unrealized Gain'].sum()
    st_unrealized = unrealized[unrealized['Type'] == 'ST']['Unrealized Gain'].sum()
    lt_unrealized = unrealized[unrealized['Type'] == 'LT']['Unrealized Gain'].sum()
    
    print(f"  Total Unrealized: ${total_unrealized:,.0f}")
    print(f"  ST Unrealized: ${st_unrealized:,.0f}")
    print(f"  LT Unrealized: ${lt_unrealized:,.0f}")
else:
    print("No remaining positions.")

In [ ]:
# Year-End Tax Planning Tool
def year_end_tax_planning(calc, current_prices, target_tax=None):
    """
    Generate year-end tax planning recommendations.
    """
    print("\nYEAR-END TAX PLANNING RECOMMENDATIONS")
    print("="*60)
    
    current_tax = calc.calculate_year_tax()
    unrealized = calc.get_unrealized(current_prices)
    
    print(f"\nCurrent Tax Situation:")
    print(f"  Estimated Cap Gains Tax: ${current_tax['total_cap_gains_tax']:,.0f}")
    
    # Find loss harvesting opportunities
    losses = unrealized[unrealized['Unrealized Gain'] < 0].copy()
    gains = unrealized[unrealized['Unrealized Gain'] > 0].copy()
    
    if len(losses) > 0:
        print(f"\n📉 LOSS HARVESTING OPPORTUNITIES:")
        print("-"*40)
        for _, row in losses.iterrows():
            loss_amt = abs(row['Unrealized Gain'])
            # Estimate tax savings
            if row['Type'] == 'ST':
                savings = loss_amt * current_tax['st_rate']
            else:
                savings = loss_amt * current_tax['lt_rate']
            
            print(f"  {row['Ticker']} Lot {row['Lot']}: ${loss_amt:,.0f} {row['Type']} loss")
            print(f"    Potential tax savings: ${savings:,.0f}")
    else:
        print(f"\n✓ No loss harvesting opportunities.")
    
    # Check for positions approaching LT
    approaching_lt = unrealized[
        (unrealized['Type'] == 'ST') & 
        (unrealized['Days Held'] > 300)
    ]
    
    if len(approaching_lt) > 0:
        print(f"\n⏰ POSITIONS APPROACHING LONG-TERM:")
        print("-"*40)
        for _, row in approaching_lt.iterrows():
            days_to_lt = 366 - row['Days Held']
            print(f"  {row['Ticker']} Lot {row['Lot']}: {days_to_lt} days until LT")
            if row['Unrealized Gain'] > 0:
                st_tax = row['Unrealized Gain'] * current_tax['st_rate']
                lt_tax = row['Unrealized Gain'] * current_tax['lt_rate']
                print(f"    Tax if sell now: ${st_tax:,.0f}")
                print(f"    Tax if wait:     ${lt_tax:,.0f}")
                print(f"    Savings:         ${st_tax - lt_tax:,.0f}")
    
    # 0% bracket opportunity
    zero_limit = 47025 if calc.filing_status == 'single' else 94050
    zero_space = max(0, zero_limit - calc.ordinary_income - current_tax['taxable_lt'])
    
    if zero_space > 0:
        lt_gains_available = gains[gains['Type'] == 'LT']
        if len(lt_gains_available) > 0:
            print(f"\n🎯 0% LTCG BRACKET OPPORTUNITY:")
            print("-"*40)
            print(f"  Space available in 0% bracket: ${zero_space:,.0f}")
            print(f"  Consider realizing LT gains tax-free to reset basis.")

year_end_tax_planning(calc, current_prices)

---

## Quiz: Week 1 Comprehensive Review (10 Questions)

In [ ]:
# Week 1 Comprehensive Quiz
quiz_questions = [
    {
        "question": "1. A taxable event for capital gains occurs when:",
        "options": ["A) Stock price increases", "B) You sell an asset", "C) You receive dividends", "D) Year-end"],
        "answer": "B",
        "explanation": "Capital gains are only taxed when you SELL and realize the gain."
    },
    {
        "question": "2. Long-term capital gains treatment requires holding for:",
        "options": ["A) 365 days", "B) More than 365 days", "C) 6 months", "D) 1 calendar year"],
        "answer": "B",
        "explanation": "More than 1 year (366+ days) qualifies for long-term treatment."
    },
    {
        "question": "3. The default cost basis method is:",
        "options": ["A) LIFO", "B) Average Cost", "C) FIFO", "D) Specific ID"],
        "answer": "C",
        "explanation": "FIFO (First-In, First-Out) is the default if you don't specify otherwise."
    },
    {
        "question": "4. Maximum capital loss deduction against ordinary income is:",
        "options": ["A) $1,000", "B) $3,000", "C) $5,000", "D) Unlimited"],
        "answer": "B",
        "explanation": "$3,000 per year ($1,500 if married filing separately)."
    },
    {
        "question": "5. The wash sale rule window is:",
        "options": ["A) 30 days", "B) 60 days", "C) 61 days (30 before + 30 after)", "D) 90 days"],
        "answer": "C",
        "explanation": "30 days before and 30 after sale = 61 day total window."
    },
    {
        "question": "6. A single filer with $40,000 income pays what LTCG rate?",
        "options": ["A) 10%", "B) 15%", "C) 20%", "D) 0%"],
        "answer": "D",
        "explanation": "Income under $47,025 qualifies for the 0% LTCG rate."
    },
    {
        "question": "7. Reinvested dividends affect cost basis by:",
        "options": ["A) No effect", "B) Creating new tax lots", "C) Reducing basis", "D) Only at sale"],
        "answer": "B",
        "explanation": "Each reinvestment creates a new lot with its own basis and holding period."
    },
    {
        "question": "8. When a wash sale occurs, the disallowed loss is:",
        "options": ["A) Lost forever", "B) Added to new shares' basis", "C) Deductible next year", "D) Converted to credit"],
        "answer": "B",
        "explanation": "Disallowed loss increases the basis of the replacement shares."
    },
    {
        "question": "9. Tax-loss harvesting is most valuable when you have:",
        "options": ["A) Only losses", "B) Short-term gains to offset", "C) No gains at all", "D) Only long-term gains"],
        "answer": "B",
        "explanation": "Offsetting ST gains saves the most tax (higher ordinary rates)."
    },
    {
        "question": "10. Capital loss carryforwards:",
        "options": ["A) Expire after 5 years", "B) Convert to ST losses", "C) Carry forward indefinitely", "D) Can only offset same type"],
        "answer": "C",
        "explanation": "Losses carry forward indefinitely, retaining their ST or LT character."
    }
]

print("WEEK 1 COMPREHENSIVE QUIZ")
print("="*50)
for i, q in enumerate(quiz_questions, 1):
    print(f"\n{q['question']}")
    for opt in q['options']:
        print(f"   {opt}")
    print(f"\n   Answer: {q['answer']} - {q['explanation']}")

---

## Summary

### Week 1 Mastery Checklist

```
✓ Investment Tax Fundamentals
  □ Three types of investment income
  □ Realized vs unrealized gains
  □ Tax deferral benefits

✓ Short-Term vs Long-Term
  □ Holding period rules (>365 days)
  □ Rate differences (ordinary vs preferential)
  □ 0% bracket opportunity

✓ Cost Basis Methods
  □ FIFO, LIFO, Specific ID, Average Cost
  □ DRIP tracking
  □ Stock split adjustments

✓ Capital Losses
  □ Netting rules (ST↔LT)
  □ $3,000 deduction limit
  □ Carryforward mechanics
  □ Wash sale avoidance
```

### What's Next?

**Week 2: Tax-Advantaged Accounts** covers:
- Traditional IRA
- Roth IRA
- 401(k) and employer plans
- HSA (Health Savings Account)
- Account selection strategies

---

### Congratulations!

You've completed **Week 1: Capital Gains**. You now understand the fundamentals of investment taxation!